In [17]:
import json
import os

import sys
sys.path.insert(0, '../src')
from transformers import DonutProcessor
from modeling_tabeleiro import TabeleiroModel

ANN_PATH = '../../aux/data/anns/train/'
IMAGE_PATH = '../../aux/data/imgs/train/'

PROCESSORS_PATH = "../../aux/processors/"
MODELS_PATH = "../../aux/models/"

IMG_FORMAT = '.png'

json_list = os.listdir(ANN_PATH)
    
json_list = [item for item in json_list]

In [18]:
aux_list = []

for json_item in json_list:
    if json_item[-6] != "L":
        aux_list.append(json_item[:-5])

json_list = aux_list

In [19]:
json_list = json_list
json_list[:10]

['PMC3549936_002_00',
 'PMC3112512_009_00',
 'PMC3772909_007_00',
 'PMC5294720_004_00',
 'PMC3446317_008_00',
 'PMC5071235_005_00',
 'PMC1421390_001_00',
 'PMC5251219_004_01',
 'PMC3973966_003_00',
 'PMC4147206_002_00']

In [20]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers import VisionEncoderDecoderConfig
import torch

config = VisionEncoderDecoderConfig.from_pretrained(MODELS_PATH+"donut-base")

image_size = [1280, 960]
max_length = 1536


config.encoder.image_size = image_size
processor = DonutProcessor.from_pretrained(PROCESSORS_PATH+"donut-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
new_tokens = ["<table_extraction>", "<table>", "<row>", "<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
new_tokens += ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            new_tokens.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            new_tokens.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


processor.tokenizer.add_tokens(new_tokens, special_tokens = False)

27

In [22]:
cell_types = ["<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


cell_tokens = [processor.tokenizer.convert_tokens_to_ids([cell_type])[0] for cell_type in cell_types]
row_tokens = [processor.tokenizer.convert_tokens_to_ids([row_type])[0] for row_type in ['<row>']]

In [23]:
processor.tokenizer.decode(row_tokens)

'<row>'

In [24]:
model = TabeleiroModel.from_pretrained("naver-clova-ix/donut-base",
                                       from_donut=True,
                                       decoder_extra_config={"pos_counters":[cell_tokens, row_tokens]},
                                       donut_config = config,
                                       ignore_mismatched_sizes=True)
model.decoder.resize_token_embeddings(len(processor.tokenizer))

Config of the decoder: <class 'modeling_dimbart.DiMBartForCausalLM'> is overwritten by shared decoder config: DiMBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_final_layer_norm": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 4,
  "dim_max_position_embeddings": [
    512,
    120,
    120
  ],
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "init_std": 0.02,
  "is_decoder": true,
  "is_encoder_decoder": false,
  "max_position_embeddings": 1536,
  "model_type": "dimbart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "pos_counters": [
    [
      57528,
      57529,
      57530,
      57531,
      57536,
      57537,
      575

Embedding(57552, 1024)

In [25]:
model.config

VisionEncoderDecoderConfig {
  "_name_or_path": "naver-clova-ix/donut-base",
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "decoder_start_token_id": null,
    "dim_max_position_embeddings": [
      512,
      120,
      120
    ],
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layerdrop": 

In [26]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
#processor.image_processor.size = {'width': image_size[::-1][0], 'height': image_size[::-1][1]}
processor.image_processor.do_align_long_axis = False


In [27]:
with open("msg.json", 'w') as out:
        json.dump({'outputs': []}, out, ensure_ascii=False, indent=4)

def write_msg(msg):
    with open("msg.json", encoding="utf-8") as f:
        json_data = json.load(f)
    
    with open("msg.json", 'w') as out:
        json_data['outputs'].append(msg)
        json.dump(json_data, out, ensure_ascii=False, indent=4)

In [28]:
def cel2token(cell):
    if cell['span_type'][10:] != '0000':
        sequence = "<" + cell['span_type'] + ">"
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<content_row_and_col_header>"
            elif cell['col_header']:
                sequence += "<content_col_header>"
            elif cell['row_header']:
                sequence += "<content_row_header>"
            else:
                sequence += "<content>"
            sequence += cell['content']
    else:
        sequence = ""
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<row_and_col_header>"
            elif cell['col_header']:
                sequence += "<col_header>"
            elif cell['row_header']:
                sequence += "<row_header>"
            else:
                sequence += "<cell>"
            sequence += cell['content']
        
    return sequence

def row2token(row):
    sequence = "<row>"
    for cell in row:
        sequence += cel2token(cell)
    
    return sequence


def table2token(table):
    sequence = "<table>"
    for row in table:
        sequence += row2token(row)
    
    return sequence


def json2token(json):
    sequence = ""
    if('tables' in json):
        for table in json['tables']:
            sequence += table2token(table)

    return sequence

In [29]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        annotations,
        image_size,
        max_length,
        shuffle = True,
        split = "train",
        ignore_id = -100,
        prompt_end_token = None,
    ):            
        self.annotations = annotations
        
        
        self.image_size = image_size
        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        
        file_name = self.annotations[idx]
        
        with open(ANN_PATH + file_name + ".json", encoding="utf-8") as f:
            annotation = json.load(f)
        
        image = Image.open(IMAGE_PATH + file_name + IMG_FORMAT)
        
        
        # inputs
        pixel_values = processor(image.convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()
        #pixel_values = processor(image.convert("RGB"), return_tensors="pt").pixel_values.squeeze()
        pixel_values = pixel_values.squeeze()
        
        target_sequence = json2token(annotation)+"</s>"
        
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length= max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id
        
        
        encoding = dict(pixel_values=pixel_values,
                        labels=labels,
                        target = target_sequence,
                       filename = file_name)
        
        return encoding

In [30]:
train_dataset = DonutTableDataset(json_list,
                             max_length = max_length,
                             image_size = image_size)

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, num_workers=2, shuffle=True)

In [16]:
checkpointed = False
if checkpointed:
    model = TabeleiroModel.from_pretrained("../../pubtabnet/checkpoints/TML-5lre-5-checkpoint")

start_epoch = 0
avg_size = 1000

In [17]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<table_extraction>"])[0]

In [ ]:
import torch
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.to(device) 
optimizer = torch.optim.AdamW(params=model.parameters(), lr=8e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=len(train_dataloader)//27, gamma=(0.125)**(1/27))

if checkpointed:
    optimizer.load_state_dict(torch.load("../../pubtabnet/checkpoints/optim-checkpoint"))

num_steps = 0   

for epoch in range(start_epoch, 2):
    
    print("Epoch:", epoch+1)
    mean_loss = 0
    mean_smpl_loss = 0 
    model.train()
    for i, batch in enumerate(tqdm(train_dataloader)):
            
        batch = {k: v.to(device) if k not in ["target", "filename"] else v for k, v in batch.items()}
        
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        
        
        loss = outputs.loss
        mean_loss += loss.item()   
        mean_smpl_loss += loss.item()
        
        loss /= 4
        loss.backward()
        
        if (i+1)%4 == 0 or i+1 == len(train_dataloader):
            optimizer.step()
            optimizer.zero_grad()
            num_steps += 1
            if num_steps%10000 == 0 :
                model.save_pretrained("../../pubtabnet/modelos/by_step/model-3D-STEP_"+str(num_steps))
        
        if  scheduler.get_last_lr()[0] > 1e-5:
            scheduler.step() 
                
        if i % avg_size == 0:
            print(str(i) + " Loss: ", mean_smpl_loss/avg_size)
            write_msg("batch " + str(i) +" loss: "+ str(mean_smpl_loss/avg_size))
            mean_smpl_loss = 0 
        
        
    
        
    model.save_pretrained("../../pubtabnet/checkpoints/TML-5lre-5-8kproc-checkpoint-epoch"+str(epoch))
    print("Epoch's mean loss: ", mean_loss/len(train_dataloader))
    
    write_msg("Epoch checkpointed: " + str(epoch+1) +" \n"+
              "Epoch's mean Loss: " + str(mean_loss/len(train_dataloader)))

Epoch: 1


  0%|          | 0/250388 [00:00<?, ?it/s]

0 Loss:  0.019431493759155274
1000 Loss:  6.579685248613358
2000 Loss:  5.570272281885147
3000 Loss:  5.2339400794506075
4000 Loss:  4.98133663058281
5000 Loss:  4.7941161029338835
6000 Loss:  4.606940300226212
7000 Loss:  4.51146121096611
8000 Loss:  4.369715010881424
9000 Loss:  4.253800024271011
10000 Loss:  4.0575041978359225
11000 Loss:  3.9440036565065384
12000 Loss:  3.7924867882728575
13000 Loss:  3.7100372223854063
14000 Loss:  3.573462166786194
15000 Loss:  3.505331085562706
16000 Loss:  3.4150975004434585
17000 Loss:  3.348776025414467
18000 Loss:  3.3226350647211076
20000 Loss:  3.20128557908535
21000 Loss:  3.1664947514533996
22000 Loss:  3.0807485001087187
23000 Loss:  3.0895066096782684
24000 Loss:  2.986276115298271
25000 Loss:  2.9030697116851805
26000 Loss:  2.815761838555336
27000 Loss:  2.719779305458069
28000 Loss:  2.5916754891872404
29000 Loss:  2.4664889006614685
30000 Loss:  2.304035267114639
31000 Loss:  2.149333081305027
32000 Loss:  1.9695193829536437
33000 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



41000 Loss:  0.7964887694567442
42000 Loss:  0.7603453059345484


...

44000 Loss:  0.081754000331508
45000 Loss:  0.08918775191996246
46000 Loss:  0.09570151437167078
47000 Loss:  0.08888829842349515
48000 Loss:  0.08998926596390083
49000 Loss:  0.09450552242109551
50000 Loss:  0.09192389869631734
51000 Loss:  0.0875583226904273
52000 Loss:  0.09080112065048888
53000 Loss:  0.0936175594984088
54000 Loss:  0.0905509135684697
55000 Loss:  0.08722285960172303
56000 Loss:  0.09153266572509892
57000 Loss:  0.08783090206515044
58000 Loss:  0.08461184310051613
59000 Loss:  0.09155658865359147
60000 Loss:  0.08666705016186461
61000 Loss:  0.09301575409597718
62000 Loss:  0.08724790156818926
63000 Loss:  0.08603669470199383
65000 Loss:  0.08271063947700895


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



74000 Loss:  0.0903660613016691


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



83000 Loss:  0.08234748573740944
84000 Loss:  0.08836262114695273
85000 Loss:  0.07854300408880226
86000 Loss:  0.09103077826381195
87000 Loss:  0.084492337531643
88000 Loss:  0.08815159448096528
89000 Loss:  0.08285651571094059
90000 Loss:  0.08768805241188966
91000 Loss:  0.09049247764749452
92000 Loss:  0.08226871493156068
93000 Loss:  0.08380051378801
94000 Loss:  0.08476647816435434
95000 Loss:  0.08603307229687926
96000 Loss:  0.07757736642390956
97000 Loss:  0.08358916757884435
98000 Loss:  0.09153812893945724


In [ ]:
processor.tokenizer.decode([57525, 57526, 57527])

In [ ]:
model.save_pretrained("../../pubtabnet/modelos/model-3D")

#### processor.save_pretrained("../../pubtabnet/processors/processor-GTML-8kproc")